In [1]:
import great_expectations as gx
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
context = gx.get_context()

spark = SparkSession \
        .builder \
        .appName("Brewery_dq") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/10 16:25:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("/opt/airflow/silver/")
dataframe_datasource = context.sources.add_or_update_spark(
    name="spark_memory"
)

In [4]:
data_asset = dataframe_datasource.add_dataframe_asset(name="dq_test", dataframe=df)

In [5]:
my_batch_request = data_asset.build_batch_request()

In [8]:
expectation_name = "null_values"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_name)


{
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.18.18"
  },
  "data_asset_type": null,
  "expectation_suite_name": "null_values",
  "ge_cloud_id": null
}

In [9]:
validator = context.get_validator(
    batch_request=my_batch_request,
    expectation_suite_name=expectation_name
)

In [10]:
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,name_address_1,name_address_2,name_address_3,brewery_type,name_city,name_country,id_brewery,latitude_brewery,longitude_brewery,name_brewery,phone_brewery,postal_code_brewery,name_state_province,name_street,website_url,dt_load,name_state
0,62970 18th St,None,None,large,Bend,United States,6d14b220-8926-4521-8d19-b98a2d6ec3db,44.08683531,-121.281706,10 Barrel Brewing Co,5415851007,97701-9847,Oregon,62970 18th St,http://www.10barrel.com,2024-06-05,Oregon
1,1135 NW Galveston Ave Ste B,None,None,large,Bend,United States,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,44.0575649,-121.3288021,10 Barrel Brewing Co,5415851007,97703-2465,Oregon,1135 NW Galveston Ave Ste B,None,2024-06-05,Oregon
2,1411 NW Flanders St,None,None,large,Portland,United States,e432899b-7f58-455f-9c7b-9a6e2130a1e0,45.5259786,-122.6855056,10 Barrel Brewing Co,5032241700,97209-2620,Oregon,1411 NW Flanders St,http://www.10barrel.com,2024-06-05,Oregon
3,62950 NE 18th St,None,None,large,Bend,United States,9f1852da-c312-42da-9a31-097bac81c4c0,44.0912109,-121.2809536,10 Barrel Brewing Co - Bend Pub,5415851007,97701,Oregon,62950 NE 18th St,None,2024-06-05,Oregon
4,141 E Main St,None,None,brewpub,John Day,United States,58293321-14ae-49d7-9a7b-08436c9e63a6,44.4146563,-118.9218754,1188 Brewing Co,5415751188,97845-1210,Oregon,141 E Main St,http://www.1188brewing.com,2024-06-05,Oregon


In [31]:
df.filter(df.name_state.isNull()).show()

+--------------+--------------+--------------+------------+---------+------------+----------+----------------+-----------------+------------+-------------+-------------------+-------------------+-----------+-----------+-------+----------+
|name_address_1|name_address_2|name_address_3|brewery_type|name_city|name_country|id_brewery|latitude_brewery|longitude_brewery|name_brewery|phone_brewery|postal_code_brewery|name_state_province|name_street|website_url|dt_load|name_state|
+--------------+--------------+--------------+------------+---------+------------+----------+----------------+-----------------+------------+-------------+-------------------+-------------------+-----------+-----------+-------+----------+
+--------------+--------------+--------------+------------+---------+------------+----------+----------------+-----------------+------------+-------------+-------------------+-------------------+-----------+-----------+-------+----------+



In [28]:
validator.expect_column_values_to_not_be_null(column="name_address_1")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false,
  "result": {
    "element_count": 50,
    "unexpected_count": 3,
    "unexpected_percent": 6.0,
    "partial_unexpected_list": [
      null,
      null,
      null
    ]
  },
  "meta": {}
}